##### Student Name : Rohan Khanolkar
##### Student Number : 13199041
##### Course : MSc. in Data Analytics (Advance Computing Technologies)
##### Birkbeck College, University of London, 
##### Date of Submission : 30th September 2021
##### Project Name: “Hybridizing a Recommendation System which incorporates collaborative and content-based filtering for an online movie streaming platform.”

###### Importing Libraries 

In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Reshape, Dropout
from keras.models import Model
from tensorflow import keras

###### Importing Ratings and Movie file from the folders

###### Please download the datafiles from the following link on Kaggle and change the folder link to the downloaded file on your computer.
###### https://www.kaggle.com/gargmanas/movierecommenderdataset 


In [2]:
r_cols_user = ['User_id','Movie_id','Rating', 'Timestamp']
user_rating_data = pd.read_csv('/Users/rohankhanolkar/Desktop/MSc_Project/report/Final_Proposal_Submission/download/ratings1.csv', skiprows = 1, sep=',', header=None, names=r_cols_user)

In [3]:
r_cols_generic = ['Movie_id','Title','Genre']
generic_additional_data = pd.read_csv('/Users/rohankhanolkar/Desktop/MSc_Project/report/Final_Proposal_Submission/download/movies1.csv', skiprows = 1, sep=',', header=None, names=r_cols_generic)

###### User Dataframe

In [4]:
user_rating_data

,User_id,Movie_id,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


###### Movie Dataframe

In [5]:
generic_additional_data

,Movie_id,Title,Genre
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


###### Split the User Rating data into train and test set. Train Data will be used to predict rating of the test data records.

In [6]:
train, test = train_test_split(user_rating_data, test_size=0.2, random_state=42, shuffle=True)

In [7]:
# Display Train data
train

,User_id,Movie_id,Rating,Timestamp
80568,509,7347,3.0,1435994597
50582,326,71462,4.0,1322252335
8344,57,2115,3.0,965798155
99603,610,1127,4.0,1479544102
71701,462,2409,2.0,1174438249
...,...,...,...,...
6265,42,4005,4.0,996259059
54886,364,141,4.0,869443367
76820,480,6867,4.0,1179163171
860,6,981,3.0,845556567


In [8]:
# Display Test data
test

,User_id,Movie_id,Rating,Timestamp
67037,432,77866,4.5,1335139641
42175,288,474,3.0,978465565
93850,599,4351,3.0,1498524542
6187,42,2987,4.0,996262677
12229,75,1610,4.0,1158989841
...,...,...,...,...
57416,380,5048,2.0,1494268065
67290,434,54272,3.5,1270606860
33423,226,5989,4.5,1162428551
98552,607,1320,3.0,963080497


#  Collaborative Filtering 

In [9]:
# Creating Utility matrix with rows as User ID and colomns as Movie ID. 
# The value in the cells represents rating given to 'j' userID by 'i' userID.
utility_matrix = train.pivot(index='User_id', columns='Movie_id', values='Rating')

In [10]:
# Display the Matrix 
utility_matrix

Movie_id,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193571,193573,193579,193581,193583,193585,193587,193609
User_id,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Filling NAN values with '0' in the Matrix
utility_matrix1 = utility_matrix.fillna(0)

In [12]:
# Display the Matrix
utility_matrix1

Movie_id,1,2,3,4,5,6,7,8,9,10,...,191005,193565,193571,193573,193579,193581,193583,193585,193587,193609
User_id,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#Find how close are the users based on the ratings provided for movies using cosine similarity on the utility matrix with itself.
df = cosine_similarity(utility_matrix1, utility_matrix1)

In [14]:
#Convert the similarity matrix into a dataframe and set the rows and column indices to the user ids
user_user_similarity = pd.DataFrame(data=df, index = utility_matrix.index, columns = utility_matrix.index,copy=True)

In [15]:
#Display user similarity dataframe
user_user_similarity

User_id,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
User_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.016314,0.049021,0.165799,0.123392,0.118556,0.112563,0.142135,0.056088,0.012906,...,0.070901,0.152097,0.187324,0.067264,0.151517,0.139042,0.198771,0.232811,0.112174,0.143902
2,0.016314,1.000000,0.000000,0.004627,0.000000,0.013391,0.029067,0.032754,0.000000,0.080739,...,0.170123,0.020395,0.014415,0.000000,0.000000,0.019846,0.016076,0.055610,0.032404,0.075810
3,0.049021,0.000000,1.000000,0.000000,0.005770,0.004833,0.000000,0.005911,0.000000,0.000000,...,0.006401,0.005889,0.015344,0.000000,0.012783,0.008884,0.004642,0.009433,0.000000,0.031309
4,0.165799,0.004627,0.000000,1.000000,0.133565,0.090914,0.094497,0.050417,0.000000,0.021991,...,0.075828,0.090252,0.241155,0.054366,0.081585,0.162277,0.083074,0.107276,0.026720,0.068325
5,0.123392,0.000000,0.005770,0.133565,1.000000,0.238812,0.071386,0.393773,0.000000,0.006245,...,0.050523,0.343953,0.101064,0.159651,0.111464,0.086797,0.073278,0.097040,0.205395,0.053090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.139042,0.019846,0.008884,0.162277,0.086797,0.086447,0.137372,0.080967,0.053366,0.061267,...,0.136437,0.077495,0.244189,0.061137,0.132016,1.000000,0.120745,0.224829,0.064349,0.159929
607,0.198771,0.016076,0.004642,0.083074,0.073278,0.135438,0.171735,0.159539,0.014172,0.012561,...,0.093158,0.158940,0.156456,0.101872,0.083353,0.120745,1.000000,0.208673,0.096324,0.097743
608,0.232811,0.055610,0.009433,0.107276,0.097040,0.136393,0.238417,0.155110,0.091135,0.051562,...,0.134926,0.141069,0.188459,0.111872,0.154623,0.224829,0.208673,1.000000,0.110371,0.260886


In [16]:
#This function calculates the predicted rating for every record in the test dataframe.
def get_ratings(test):
    #Declare a pred array to return as result
    pred = []
    #Iterate through the test records and for each record do the following.
    for index, row in test.iterrows():  
        #Get user id of the test record
        u_id = row['User_id']
        #Get movie id of the test record
        m_id = row['Movie_id']
        #If the user similarity dataframe contains a record of the user (Handling the cold start problem)
        if u_id in user_user_similarity.columns:
            #Declare a dictionary of the cosine similarity of the test record user with the other users
            dfdict = user_user_similarity.loc[:,u_id].to_dict()
            #Sort the dictionary in the descending order such that the users closest to the users are placed at the top.
            sorted_d = dict(sorted(dfdict.items(),key=operator.itemgetter(1),reverse=True))
            #Slice out the closest 200 users
            out = dict(itertools.islice(sorted_d.items(), 20))
            #Get the user ids of the sliced items
            topusers=list(out.keys())
            #Get user rating subset dataframe of the users in the topusers list
            dfsubset=user_rating_data[user_rating_data['User_id'].isin(topusers)]
            #Filter out the user records which have not watched the movie in the test record
            dfsubset=dfsubset[dfsubset['Movie_id']==m_id]
            #Calculate Mean of the ratings (Users who have watched the test record movie) and append it to the pred array
            pred.append(dfsubset['Rating'].mean())
        else:
            #If the user is new (cold start), append 0 to the pred array
            pred.append(0)
    #Return the pred array consisting of the predicted ratings of the test dataframe
    return pred

In [17]:
#Call get_ratings function on the test dataframe
pred = get_ratings(test)
#Backup
pred_bkp = pred
len(pred)

20168

In [18]:
#This is the pred array
pred

[4.5,
 3.35,
 3.0,
 3.8125,
 3.625,
 3.5,
 4.083333333333333,
 4.277777777777778,
 2.5,
 3.9583333333333335,
 4.75,
 4.214285714285714,
 3.7916666666666665,
 4.1,
 3.0,
 3.642857142857143,
 4.5,
 3.0,
 4.0,
 2.75,
 3.730769230769231,
 2.0,
 4.1875,
 3.85,
 4.026315789473684,
 3.1666666666666665,
 2.0,
 3.375,
 3.5,
 4.0,
 3.0,
 4.5,
 3.3,
 4.4,
 3.1666666666666665,
 3.5,
 3.1666666666666665,
 4.03125,
 3.4615384615384617,
 4.0,
 3.2142857142857144,
 3.5,
 3.6333333333333333,
 3.6666666666666665,
 4.35,
 2.8333333333333335,
 4.0,
 4.25,
 3.3333333333333335,
 4.617647058823529,
 3.3125,
 3.125,
 4.633333333333334,
 3.125,
 3.875,
 4.166666666666667,
 3.0,
 4.428571428571429,
 3.5555555555555554,
 3.5454545454545454,
 4.071428571428571,
 2.0,
 3.0,
 2.8181818181818183,
 2.0,
 3.0714285714285716,
 2.5,
 2.96875,
 3.75,
 4.0,
 3.3333333333333335,
 3.5,
 2.5,
 4.0,
 3.5,
 4.0,
 4.214285714285714,
 4.5,
 1.6666666666666667,
 3.5833333333333335,
 2.772727272727273,
 3.0,
 3.125,
 4.5,
 3.4,
 4

In [19]:
#This function calculates the predicted rating for every record in the test dataframe Using Weighted Average to improve accuracy.
def get_ratings(test):
    #Declare a pred array to return as result
    pred = []
    #Iterate through the test records and for each record do the following.
    for index, row in test.iterrows():  
        #Get user id of the test record
        u_id = row['User_id']
        #Get movie id of the test record
        m_id = row['Movie_id']
        #If the user similarity dataframe contains a record of the user (Handling the cold start problem)
        if u_id in user_user_similarity.columns:
            #Declare a dictionary of the cosine similarity of the test record user with the other users
            dfdict = user_user_similarity.loc[:,u_id].to_dict()
            #Sort the dictionary in the descending order such that the users closest to the users are placed at the top.
            sorted_d = dict(sorted(dfdict.items(),key=operator.itemgetter(1),reverse=True))
            #Slice out the closest 200 users
            out = dict(itertools.islice(sorted_d.items(), 20))
            #Get the user ids of the sliced items
            topusers=list(out.keys())
            #Get user rating subset dataframe of the users in the topusers list
            dfsubset=user_rating_data[user_rating_data['User_id'].isin(topusers)]
            #Filter out the user records which have not watched the movie in the test record
            dfsubset=dfsubset[dfsubset['Movie_id']==m_id]
            #Calculate Mean of the ratings (Users who have watched the test record movie) and append it to the pred array
            #pred.append(dfsubset['Rating'].mean())
            #Calculate sum of the similarities of top 20 users
            test = sum([out[x] for x in list(dfsubset['User_id'])])
            #Initialize rating to 0
            rating = 0
            #For every similar user who has rated the movie, add all the weighted ratings (Rating * similarity)
            for index, row in dfsubset.iterrows():
                rating = rating + (out.get(int(row['User_id'])) * row['Rating'])
            #Calculate the average weighted rating and append to the pred
            if(test!=0):
                predrating = rating / test
                pred.append(predrating)
            else: 
                pred.append(np.nan)
        else:
            #If the user is new (cold start), append 0 to the pred array
            pred.append(0)
    #Return the pred array consisting of the predicted ratings of the test dataframe
    return pred

In [20]:
#Call get_ratings function on the test dataframe
pred2 = get_ratings(test)
#Backup
pred_bkp2 = pred2
len(pred2)

20168

In [21]:
#This is the pred array
pred2

[4.5,
 3.2884165093684787,
 3.0327945317497558,
 3.8387289472862194,
 3.8067427246596703,
 3.840593879477376,
 3.988643181260228,
 4.347602115388511,
 2.150445833473403,
 3.865396362648509,
 4.657188788454478,
 4.171182207467544,
 3.971000025330138,
 4.373930809314643,
 3.672790401947117,
 3.637492901400351,
 4.5,
 2.9999999999999996,
 3.9394695648397597,
 2.701793920821945,
 3.7155208793513896,
 2.0,
 4.272823820122051,
 3.8629911646829345,
 4.031970786714925,
 3.106045889190588,
 2.0,
 2.8919944646661433,
 3.3500188562001463,
 4.0,
 3.0,
 4.5,
 2.4969247460074246,
 4.2275813499978865,
 3.3458203640440733,
 3.833695022264884,
 2.672099299199395,
 4.045404836620845,
 3.484264527424324,
 4.0,
 3.301825885742618,
 3.238840761973021,
 3.799171938297888,
 3.3363512750424507,
 4.423553946324221,
 2.920206437306699,
 4.0,
 4.193698514032512,
 3.5428684162438495,
 4.516244398872945,
 3.5869321713015134,
 3.083256811759517,
 4.688543412354761,
 2.710709961034497,
 3.766371848185337,
 4.1016758

# Content based filtering using the movie.csv data

In [22]:
#This function is used to predict ratings based on content preference of the test record user. 
#It calculates rating based on the users preference of movies based on movie genres.
def get_preference_ratings(test):
    #Declare a pred_content array to return as result
    pred_content = []
    #Iterate through the test records and for each record do the following.
    for index, row in test.iterrows():
        #Get a user rating dataframe subset of movies that the user has watched
        user_watch_movies = user_rating_data[user_rating_data.User_id==row['User_id']]
        #Declare a dictionary of movies that the user has watched to save the genres of that movie
        user_movie_dict = {}
        #For every movie watched by the user do the following
        for index, row in user_watch_movies.iterrows():
            #Get a list of movies and its respective genres from the movies dataframe
            additional_data_list = list(generic_additional_data.loc[generic_additional_data['Movie_id'] == row['Movie_id'],'Genre'])
            #Fill in the dictionary such that key is the movie id and value is the movie genre
            user_movie_dict[int(row['Movie_id'])] = additional_data_list
        #Convert above dictionary into dataframe
        user_movie_df = pd.DataFrame.from_dict(user_movie_dict, orient='index')
        #Remove the pipe separator in the genres column and replace it by a space for tf-idf use purpose
        user_movie_df['Merged'] = user_movie_df[0].apply(lambda x: ' '.join(str(x).split('|')))
        user_movie_df = user_movie_df[['Merged']]
        #Vectorize the movie records in the user_movie_df
        X = vectorizer.fit_transform(user_movie_df['Merged'])
        #Create a dictionary for the test record movie
        user_movie_dict_predict = {}
        #Get the movie and its respective genre from the movies dataframe
        additional_data_list = list(generic_additional_data.loc[generic_additional_data['Movie_id'] == row['Movie_id'],'Genre'])
        #Fill in the dictionary such that key is the movie id and value is the movie genre
        user_movie_dict_predict[int(row['Movie_id'])] = additional_data_list
        #Convert above dictionary into dataframe
        user_movie_predict_df = pd.DataFrame.from_dict(user_movie_dict_predict, orient='index')
        #Remove the pipe separator in the genres column and replace it by a space for tf-idf use purpose
        user_movie_predict_df['Merged'] = user_movie_predict_df[0].apply(lambda x: ' '.join(str(x).split('|')))
        user_movie_predict_df = user_movie_predict_df[['Merged']]
        #Vectorize the movie record in the user_movie_predict_df
        X_test = vectorizer.transform(user_movie_predict_df['Merged'])
        #Calculate how close the test movie is from other movies that the user has watched using cosine similarity
        df_movie_similarity = cosine_similarity(X, X_test)
        #Convert above similarity matrix into  a dataframe such that the rows are user watched movies and the column is the test movie
        df_movie_similarity = pd.DataFrame(data=df_movie_similarity, index = user_movie_df.index,copy=True)
        #Convert the dataframe to dictionary such that key is the movie id and value is the cosine similarity value 
        dfdict=df_movie_similarity.loc[:,0].to_dict()
        #Sort the dictionary in the descending order such that the user watched movies closest to the test movie are placed at the top.
        sorted_df = dict(sorted(dfdict.items(), key=operator.itemgetter(1),reverse=True))
        #Slice out the closest 5 user watched movies
        topmovies=list(dict(itertools.islice(sorted_df.items(), 5)).keys())
        #Get user rating subset dataframe of the movies in the topmovies list
        dfsubset=user_rating_data[user_rating_data['Movie_id'].isin(topmovies)]
        #Calculate Mean of the ratings and append it to the pred array
        p_content=dfsubset['Rating'].mean()
        pred_content.append(p_content)
        #Return pred_count array
    return pred_content

In [23]:
#Call get_reference_ratings function on the test data to predict content-based filtertering ratings.
# it takes 2hr 30 min to process this line of code.
pred_content = get_preference_ratings(test)

In [24]:
pred_content

[3.743362831858407,
 3.5053908355795147,
 3.361111111111111,
 3.207920792079208,
 3.582379862700229,
 3.2560975609756095,
 3.473404255319149,
 3.8313008130081303,
 3.6892857142857145,
 3.8454810495626823,
 3.2263157894736842,
 4.297794117647059,
 3.5510204081632653,
 3.393238434163701,
 4.089700996677741,
 3.9118773946360155,
 3.090551181102362,
 3.7378048780487805,
 3.3333333333333335,
 3.0486486486486486,
 3.6991150442477876,
 3.0480225988700567,
 4.2290448343079925,
 3.8123359580052494,
 3.6417624521072796,
 2.9746835443037973,
 4.154220779220779,
 3.7608142493638677,
 3.8801369863013697,
 3.28125,
 4.23440643863179,
 3.7713414634146343,
 3.7772861356932155,
 3.9297297297297296,
 3.85,
 3.9338235294117645,
 3.7901960784313724,
 3.650273224043716,
 3.0486486486486486,
 3.6417624521072796,
 3.7432432432432434,
 4.0908203125,
 3.5899419729206965,
 3.91497975708502,
 3.9774266365688487,
 3.5027777777777778,
 4.146844660194175,
 3.697309417040359,
 3.5886699507389164,
 3.5063391442155307

##### Neural Network ML model

In [25]:
# Create an embedding layer for Movie Input.  
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(193609+1, 7, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Books")(movie_embedding)

# # Create an embedding layer for User Input.
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(610+1, 7, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

# concatenate both the embedding layers
conc = Concatenate()([movie_vec, user_vec])
# We add the hidden layers and output layers (i.e. fully-connected-layers)
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

# Create a model and compile the model 
model2 = Model([user_input, movie_input], out)
model2.compile('adam', 'mean_squared_error')

In [26]:
#Fit the train data
history = model2.fit([train.User_id, train.Movie_id], train.Rating, epochs=5, verbose=1)


Epoch 1/5
2521/2521 [==============================] - 38s 14ms/step - loss: 1.7869
Epoch 2/5
2521/2521 [==============================] - 35s 14ms/step - loss: 0.7004
Epoch 3/5
2521/2521 [==============================] - 35s 14ms/step - loss: 0.6480
Epoch 4/5
2521/2521 [==============================] - 35s 14ms/step - loss: 0.6272
Epoch 5/5
2521/2521 [==============================] - 51s 20ms/step - loss: 0.6047


In [27]:
# Predict on the Test data and print the predictions
predictions = model2.predict([test.User_id, test.Movie_id])
[print(predictions[i], test.Rating.iloc[i]) for i in range(0,len(predictions))]

[3.198333] 4.5
[3.5888093] 3.0
[2.4247172] 3.0
[3.8369985] 4.0
[3.3389008] 4.0
[3.1355987] 4.0
[3.927942] 3.5
[3.6151416] 4.5
[2.8843215] 0.5
[3.8275354] 3.5
[3.0582123] 4.5
[3.583655] 4.0
[4.082748] 5.0
[4.337361] 5.0
[3.620274] 4.0
[3.2377503] 3.5
[3.339966] 4.5
[3.35734] 3.0
[3.9419968] 3.5
[2.97836] 2.5
[4.023118] 3.5
[3.5064511] 2.0
[3.8073308] 5.0
[4.356571] 4.0
[4.142782] 4.0
[3.873246] 3.0
[1.8839794] 2.0
[3.465706] 1.5
[3.314794] 3.0
[3.6532052] 4.0
[3.2147815] 3.0
[3.562901] 4.5
[3.9126236] 1.5
[3.6090195] 4.0
[3.6365068] 3.5
[3.5647585] 4.0
[3.6993086] 2.0
[3.9625218] 4.0
[3.6756775] 3.5
[3.9522834] 4.0
[3.0647209] 3.5
[1.716415] 3.0
[4.1409745] 5.0
[3.6834018] 3.0
[3.8951604] 4.5
[2.878897] 3.0
[1.5264293] 4.0
[4.0644107] 4.0
[4.339606] 4.0
[4.2844543] 4.0
[4.110813] 4.5
[3.9059174] 3.0
[4.3936496] 5.0
[1.4706017] 1.5
[3.4281971] 3.5
[3.766987] 4.0
[3.231723] 3.0
[4.3695545] 4.0
[3.5657613] 4.0
[2.6532586] 2.5
[4.2442017] 4.5
[3.3593957] 0.5
[2.245826] 3.0
[2.6661675] 3.0
[

[1.8899063] 1.5
[3.9610517] 2.5
[4.042919] 4.5
[3.6801593] 2.5
[3.5436323] 4.0
[3.60544] 4.0
[3.306399] 4.0
[3.3334222] 2.0
[3.3065403] 2.0
[1.7555913] 3.0
[4.1833873] 3.0
[3.4918427] 3.0
[2.4770157] 3.0
[3.7812493] 4.0
[3.0905635] 3.0
[4.3491797] 5.0
[3.618633] 3.5
[2.542926] 2.0
[4.173582] 4.0
[4.12431] 4.0
[3.4034007] 3.0
[2.2346225] 3.0
[3.003679] 2.0
[3.951975] 3.5
[2.3288147] 3.5
[3.8985817] 4.5
[2.8015854] 3.0
[4.3679633] 4.5
[3.4363277] 4.5
[4.388223] 4.0
[3.8816392] 3.5
[3.3956754] 4.0
[3.855829] 3.5
[2.0299776] 2.5
[4.2633324] 5.0
[3.3248482] 2.0
[3.3423088] 4.5
[2.8472733] 3.0
[2.0010931] 4.0
[2.492695] 2.5
[3.6468463] 3.5
[4.4936333] 4.0
[2.5894945] 2.0
[3.5999048] 3.0
[4.3565044] 5.0
[3.3788233] 4.0
[3.3091037] 3.0
[4.4784575] 4.0
[3.0929978] 1.0
[3.5516088] 2.5
[4.1682277] 5.0
[4.4773207] 4.5
[3.673354] 4.0
[3.8940673] 3.0
[4.2944956] 4.0
[2.8325083] 4.0
[4.022761] 4.0
[4.2286386] 5.0
[3.9314504] 4.0
[3.9528015] 4.0
[3.5916042] 3.5
[3.9507468] 4.5
[3.8030581] 4.0
[3.78046

[3.709059] 5.0
[4.2928934] 5.0
[3.5990024] 3.5
[3.7610557] 4.0
[2.8224783] 3.5
[3.2325284] 2.5
[4.0094047] 3.0
[3.2538517] 4.0
[3.0169685] 3.0
[3.7133534] 3.0
[3.0367088] 3.0
[3.9155807] 3.0
[3.6606672] 3.0
[3.900263] 3.0
[3.0996726] 3.5
[3.066835] 3.0
[3.8282769] 5.0
[2.5788474] 1.0
[3.5888956] 4.5
[3.9200897] 3.5
[3.5332558] 4.0
[3.5367434] 1.0
[3.3406744] 3.5
[3.739632] 3.0
[3.4117048] 2.0
[3.7766397] 5.0
[3.7999854] 4.0
[3.189446] 4.0
[3.116228] 3.5
[2.92254] 1.0
[2.3676636] 2.0
[2.2789931] 2.5
[1.7103668] 2.5
[4.1597915] 4.5
[2.9809954] 3.0
[2.856962] 2.0
[4.5478077] 5.0
[2.8183494] 2.5
[4.328891] 4.0
[4.218418] 4.5
[2.5325363] 3.0
[3.1088073] 4.0
[3.972196] 3.0
[3.1865268] 3.5
[3.3644145] 2.0
[3.0680606] 2.0
[4.0119195] 4.5
[3.896106] 3.5
[3.854503] 5.0
[3.4568527] 1.0
[4.086653] 5.0
[3.6475863] 3.5
[3.999551] 5.0
[2.849343] 4.0
[3.7653544] 2.5
[3.4084485] 1.0
[3.6317203] 4.0
[3.8850257] 4.0
[3.4019773] 5.0
[3.424308] 2.0
[3.6644027] 3.5
[3.271374] 0.5
[4.2402015] 2.5
[3.9422128]

[3.8230135] 4.5
[3.899359] 2.5
[3.456651] 3.5
[2.753155] 3.0
[4.119218] 3.0
[3.9012825] 4.0
[1.4970714] 1.5
[2.2514355] 3.0
[4.1150446] 3.0
[4.003951] 4.0
[4.365231] 4.0
[3.534889] 4.0
[3.0490112] 2.0
[3.8716018] 2.5
[4.0510926] 5.0
[4.218167] 4.5
[3.3008964] 3.0
[3.5020838] 3.0
[3.7033632] 4.0
[3.7747934] 2.0
[2.7242305] 3.0
[3.385999] 4.0
[2.3727655] 3.5
[3.5176182] 4.0
[3.8812249] 4.0
[3.4717681] 4.0
[4.0177417] 3.5
[4.1640987] 4.0
[4.3473616] 4.0
[2.890228] 2.5
[2.4266703] 3.0
[3.4903958] 3.0
[4.205322] 4.0
[4.2604713] 5.0
[3.0231142] 3.0
[3.3082886] 3.5
[1.8778286] 2.5
[3.0039022] 3.0
[3.3078916] 4.0
[4.057783] 4.0
[2.543152] 3.5
[3.413624] 4.0
[1.2075279] 4.0
[4.1222606] 5.0
[3.7856462] 2.5
[4.837198] 5.0
[2.9638433] 4.0
[2.830918] 3.0
[3.435122] 3.0
[4.3235745] 4.0
[3.5195525] 5.0
[2.8783529] 1.5
[4.1083636] 5.0
[2.9596786] 1.0
[3.9071643] 4.0
[1.2037319] 3.0
[3.5943148] 4.0
[3.5134785] 3.5
[3.0562806] 3.0
[3.3472989] 3.0
[3.7484457] 3.5
[3.709408] 4.5
[4.2967577] 5.0
[4.0062685

[3.8626456] 3.5
[2.4366663] 2.5
[3.9147546] 4.0
[3.4890277] 4.0
[2.2948472] 1.0
[3.9955924] 5.0
[4.0572314] 2.5
[4.154954] 4.5
[3.968006] 4.0
[3.5487328] 5.0
[3.720226] 3.0
[4.2256255] 4.0
[4.0680494] 4.5
[2.1946542] 3.5
[3.0088055] 4.0
[4.0255523] 4.0
[4.0098248] 5.0
[2.7379537] 1.5
[3.941859] 4.5
[4.065974] 4.5
[2.3198578] 3.0
[3.3206782] 2.5
[3.6534317] 4.5
[3.3469963] 4.0
[3.7943366] 3.0
[2.9084632] 4.0
[4.203722] 5.0
[3.4494364] 4.0
[3.9630792] 4.0
[3.9789708] 5.0
[2.6791418] 4.0
[3.8586285] 3.5
[3.772208] 4.0
[3.998634] 4.0
[3.6341066] 4.0
[3.1562583] 2.0
[4.366595] 5.0
[3.5692623] 5.0
[2.4799912] 3.0
[3.5695755] 3.0
[4.2008467] 4.0
[4.0035033] 4.0
[2.2753654] 2.0
[3.015526] 3.5
[3.3868344] 3.5
[3.030885] 3.0
[1.3547161] 0.5
[3.2910454] 4.0
[3.6707547] 3.0
[3.9419348] 4.0
[3.754341] 4.0
[2.562306] 2.0
[3.954933] 3.5
[3.2737734] 3.5
[3.8328474] 4.0
[4.3428583] 4.0
[3.8260586] 2.0
[3.7961667] 3.5
[2.5497468] 4.0
[4.3273163] 5.0
[4.015471] 5.0
[3.2531517] 0.5
[2.9584754] 4.0
[3.4785

...

[3.8563008] 4.0
[3.42467] 2.0
[3.2931666] 3.0
[3.7278454] 3.5
[2.8596342] 5.0
[3.514148] 4.0
[3.651261] 3.0
[3.6708515] 4.0
[3.8283517] 0.5
[4.2622623] 3.0
[2.6350663] 3.5
[1.6366343] 1.0
[3.191497] 4.0
[3.1155777] 3.5
[4.34713] 5.0
[3.3007886] 3.0
[3.7119272] 4.0
[3.5869095] 3.5
[4.317704] 5.0
[4.303354] 5.0
[4.0758014] 5.0
[3.2793658] 2.0
[4.6607695] 4.0
[3.261696] 3.0
[2.9390402] 3.0
[2.9340146] 3.5
[4.034171] 2.0
[4.728506] 4.5
[3.286166] 4.0
[2.6623836] 3.5
[3.3146966] 2.5
[2.3271444] 1.5
[3.6314878] 5.0
[3.7523835] 3.0
[3.9568565] 5.0
[3.042195] 3.5
[3.9889832] 4.0
[3.5265863] 2.0
[3.9270031] 3.5
[4.2284527] 4.0
[3.7857249] 5.0
[3.1010392] 3.0
[3.3526495] 4.0
[3.9977221] 1.5
[4.0455995] 4.0
[1.9310385] 2.0
[2.8479226] 3.0
[1.7776567] 2.0
[4.3706236] 4.5
[4.3376412] 3.0
[3.4727929] 4.0
[3.256784] 4.0
[3.852104] 4.0
[2.9843872] 3.5
[3.55464] 2.0
[3.4484105] 3.0
[3.0920918] 3.0
[3.6483157] 5.0
[3.7192166] 3.5
[4.4194493] 4.5
[3.415097] 4.0
[1.6833917] 1.5
[3.668104] 3.5
[3.092088] 3

[4.2831707] 4.0
[3.1015604] 4.0
[3.7949693] 3.5
[3.3904178] 3.0
[3.0241034] 3.5
[3.2425349] 2.5
[3.8550165] 3.5
[1.7404004] 2.0
[3.7673934] 4.0
[4.150597] 4.0
[3.1399786] 3.0
[2.5815918] 3.0
[3.7998502] 3.5
[3.3168452] 3.0
[4.3655486] 5.0
[2.928204] 3.0
[3.2438023] 4.0
[4.146758] 4.0
[3.7632768] 4.0
[4.0386744] 3.5
[3.9724925] 2.5
[3.6518247] 4.0
[3.0199952] 3.0
[3.6658552] 3.5
[3.317538] 4.0
[4.074497] 5.0
[3.4415114] 3.0
[3.481454] 2.0
[3.7235053] 4.0
[2.8503916] 2.5
[3.6820142] 3.5
[3.035165] 2.5
[4.0884123] 4.0
[4.0133386] 4.0
[3.2863705] 3.5
[4.5047703] 4.5
[3.8989198] 4.5
[3.8273804] 4.5
[3.7203856] 4.0
[2.8034043] 3.5
[3.3061078] 3.5
[3.4516647] 4.5
[4.123507] 3.0
[3.9410887] 4.0
[3.6420484] 4.0
[3.4811924] 3.5
[3.5239084] 3.5
[3.7464523] 4.5
[3.0330594] 3.5
[4.1301556] 5.0
[3.4781523] 3.5
[2.2135007] 2.5
[3.0194376] 2.5
[4.241463] 5.0
[2.6256032] 2.5
[2.4624522] 2.0
[3.808972] 4.0
[3.693079] 4.5
[2.913466] 3.0
[3.4180815] 4.0
[3.8895874] 4.0
[3.8163457] 4.0
[3.5661433] 3.5
[4.4

[4.3635073] 5.0
[3.7068367] 3.5
[4.216502] 5.0
[2.965543] 3.0
[1.6905669] 2.0
[2.0903416] 3.0
[3.305589] 1.0
[3.6146743] 4.0
[3.9621522] 5.0
[2.436585] 3.0
[3.4978478] 3.5
[3.6663268] 4.5
[3.515607] 5.0
[3.5299008] 3.5
[4.0562716] 4.0
[4.1593785] 3.0
[3.4164326] 2.0
[3.8924553] 3.0
[3.434537] 4.0
[2.5419252] 1.0
[3.7896416] 4.0
[4.12344] 4.5
[3.524363] 4.0
[3.748029] 2.5
[3.5222855] 4.0
[4.366214] 5.0
[4.421838] 5.0
[3.9880035] 5.0
[2.4429212] 3.0
[4.2166443] 3.0
[4.1864595] 5.0
[3.522586] 4.0
[3.5960093] 3.5
[2.9217708] 3.0
[1.721018] 1.0
[3.5428863] 4.0
[3.360477] 4.0
[3.8125217] 4.0
[3.7456977] 5.0
[2.1356957] 2.5
[3.9151468] 4.5
[3.1387587] 3.0
[4.120767] 4.0
[2.1190789] 3.0
[3.3632686] 2.0
[4.4011354] 4.5
[1.9173394] 2.5
[3.7940114] 2.5
[3.575587] 5.0
[3.5039155] 3.5
[3.694481] 4.0
[2.972537] 0.5
[3.9067194] 3.0
[4.1898565] 5.0
[2.8265293] 1.5
[2.943199] 4.5
[3.2561824] 3.5
[3.0620234] 3.5
[4.419082] 5.0
[3.69473] 1.5
[3.1065547] 3.0
[3.150544] 3.0
[4.4905043] 5.0
[4.6912065] 5.0


[3.5735288] 4.0
[4.7135715] 4.0
[3.7689664] 3.0
[2.6856406] 3.0
[3.1043174] 2.0
[4.110813] 3.0
[2.7382147] 2.0
[3.0624597] 1.5
[3.272023] 3.0
[2.9451158] 4.0
[3.9059694] 5.0
[4.4820123] 3.5
[4.122802] 5.0
[3.9351752] 4.0
[3.6257656] 4.5
[4.236368] 4.0
[3.0088995] 1.0
[3.2501962] 3.0
[3.703284] 4.5
[3.0608432] 3.0
[2.8506105] 1.5
[4.1175175] 3.0
[4.3216143] 4.5
[3.6489737] 3.5
[3.493253] 2.0
[1.8055664] 2.0
[4.564481] 3.0
[3.4249585] 4.0
[4.007319] 3.0
[3.662832] 2.5
[3.7035234] 3.5
[3.1488764] 3.5
[4.880163] 3.0
[2.6258433] 3.0
[3.1106057] 4.0
[2.5666196] 2.5
[4.375345] 5.0
[4.1708336] 4.5
[3.4006717] 4.0
[3.967839] 5.0
[3.7223256] 4.0
[3.2664135] 4.0
[4.0727654] 4.0
[3.6962042] 4.0
[3.5037944] 4.5
[4.026647] 4.0
[3.3097236] 3.0
[3.4865472] 4.0
[3.28932] 4.0
[2.4529295] 2.0
[2.7614882] 2.0
[2.30408] 2.0
[4.3432817] 4.5
[3.4022543] 4.0
[3.856069] 3.5
[3.71993] 5.0
[2.9597867] 2.0
[2.9481544] 3.0
[3.5611286] 3.5
[3.852164] 4.0
[3.4405613] 4.5
[3.4712713] 4.0
[3.6309135] 4.0
[1.9170517] 0

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [28]:
#Add CB_pred column to the test dataframe. These are the predicted ratings by the users for the respective movies
test['CB_Pred'] = pred

In [29]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred
67037,432,77866,4.5,1335139641,4.500000
42175,288,474,3.0,978465565,3.350000
93850,599,4351,3.0,1498524542,3.000000
6187,42,2987,4.0,996262677,3.812500
12229,75,1610,4.0,1158989841,3.625000
...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000
67290,434,54272,3.5,1270606860,3.625000
33423,226,5989,4.5,1162428551,3.852941
98552,607,1320,3.0,963080497,3.045455


In [30]:
#Add Content_Pred column to the test dataframe. These are the predicted ratings by the users for the respective movies
test['Content_Pred'] = pred_content

In [31]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred
67037,432,77866,4.5,1335139641,4.500000,3.743363
42175,288,474,3.0,978465565,3.350000,3.505391
93850,599,4351,3.0,1498524542,3.000000,3.361111
6187,42,2987,4.0,996262677,3.812500,3.207921
12229,75,1610,4.0,1158989841,3.625000,3.582380
...,...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000,3.551020
67290,434,54272,3.5,1270606860,3.625000,3.990000
33423,226,5989,4.5,1162428551,3.852941,3.668297
98552,607,1320,3.0,963080497,3.045455,4.114672


In [32]:
#Add CB_pred column to the test dataframe. These are the predicted ratings by the users for the respective movies
test['CB_Pred2'] = pred2

In [33]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred,CB_Pred2
67037,432,77866,4.5,1335139641,4.500000,3.743363,4.500000
42175,288,474,3.0,978465565,3.350000,3.505391,3.288417
93850,599,4351,3.0,1498524542,3.000000,3.361111,3.032795
6187,42,2987,4.0,996262677,3.812500,3.207921,3.838729
12229,75,1610,4.0,1158989841,3.625000,3.582380,3.806743
...,...,...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000,3.551020,2.000000
67290,434,54272,3.5,1270606860,3.625000,3.990000,3.615817
33423,226,5989,4.5,1162428551,3.852941,3.668297,3.934037
98552,607,1320,3.0,963080497,3.045455,4.114672,3.036918


In [34]:
test['NN_Pred'] = predictions

In [35]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred,CB_Pred2,NN_Pred
67037,432,77866,4.5,1335139641,4.500000,3.743363,4.500000,3.198333
42175,288,474,3.0,978465565,3.350000,3.505391,3.288417,3.588809
93850,599,4351,3.0,1498524542,3.000000,3.361111,3.032795,2.424717
6187,42,2987,4.0,996262677,3.812500,3.207921,3.838729,3.836998
12229,75,1610,4.0,1158989841,3.625000,3.582380,3.806743,3.338901
...,...,...,...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000,3.551020,2.000000,3.254596
67290,434,54272,3.5,1270606860,3.625000,3.990000,3.615817,3.494291
33423,226,5989,4.5,1162428551,3.852941,3.668297,3.934037,3.901221
98552,607,1320,3.0,963080497,3.045455,4.114672,3.036918,3.591904


##### Check for NaN values or missiing values.

In [36]:
#Get function for the Nan predicted ratings
def get_Nan(user_test_data):   
    #Check if any null values are present
    is_NaN = user_test_data.isnull()
    #Get row indices with Nan value in any column of the test dataframe
    row_has_NaN = is_NaN.any(axis=1)
    #Get test dataframe subset with predicted ratings as Nan
    rows_with_NaN = user_test_data[row_has_NaN]
    #Return dataframe
    return rows_with_NaN

In [37]:
#Call the get_Nan function
test_nan = get_Nan(test)

In [38]:
test_nan

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred,CB_Pred2,NN_Pred


In [39]:
#Calculate mean of the CB_pred and Content_Pred ratings
test['CB_CP_Mean_Pred'] = test.loc[: , "CB_Pred":"Content_Pred"].mean(axis=1)

In [40]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred,CB_Pred2,NN_Pred,CB_CP_Mean_Pred
67037,432,77866,4.5,1335139641,4.500000,3.743363,4.500000,3.198333,4.121681
42175,288,474,3.0,978465565,3.350000,3.505391,3.288417,3.588809,3.427695
93850,599,4351,3.0,1498524542,3.000000,3.361111,3.032795,2.424717,3.180556
6187,42,2987,4.0,996262677,3.812500,3.207921,3.838729,3.836998,3.510210
12229,75,1610,4.0,1158989841,3.625000,3.582380,3.806743,3.338901,3.603690
...,...,...,...,...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000,3.551020,2.000000,3.254596,2.775510
67290,434,54272,3.5,1270606860,3.625000,3.990000,3.615817,3.494291,3.807500
33423,226,5989,4.5,1162428551,3.852941,3.668297,3.934037,3.901221,3.760619
98552,607,1320,3.0,963080497,3.045455,4.114672,3.036918,3.591904,3.580063


In [41]:
#Calculate mean of the Content_Pred and CB_Pred2 ratings
test['CB2_CP_Mean_Pred'] = test.loc[: , "Content_Pred":"CB_Pred2"].mean(axis=1)

In [42]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred,CB_Pred2,NN_Pred,CB_CP_Mean_Pred,CB2_CP_Mean_Pred
67037,432,77866,4.5,1335139641,4.500000,3.743363,4.500000,3.198333,4.121681,4.121681
42175,288,474,3.0,978465565,3.350000,3.505391,3.288417,3.588809,3.427695,3.396904
93850,599,4351,3.0,1498524542,3.000000,3.361111,3.032795,2.424717,3.180556,3.196953
6187,42,2987,4.0,996262677,3.812500,3.207921,3.838729,3.836998,3.510210,3.523325
12229,75,1610,4.0,1158989841,3.625000,3.582380,3.806743,3.338901,3.603690,3.694561
...,...,...,...,...,...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000,3.551020,2.000000,3.254596,2.775510,2.775510
67290,434,54272,3.5,1270606860,3.625000,3.990000,3.615817,3.494291,3.807500,3.802909
33423,226,5989,4.5,1162428551,3.852941,3.668297,3.934037,3.901221,3.760619,3.801167
98552,607,1320,3.0,963080497,3.045455,4.114672,3.036918,3.591904,3.580063,3.575795


In [43]:
#Calculate Overall mean of the Content_Pred, CB_Pred2, CB_Pred and NN_Pred ratings
test['Overall_Mean_Pred'] = test.loc[: , "CB_Pred":"NN_Pred"].mean(axis=1)

In [44]:
test

,User_id,Movie_id,Rating,Timestamp,CB_Pred,Content_Pred,CB_Pred2,NN_Pred,CB_CP_Mean_Pred,CB2_CP_Mean_Pred,Overall_Mean_Pred
67037,432,77866,4.5,1335139641,4.500000,3.743363,4.500000,3.198333,4.121681,4.121681,3.985424
42175,288,474,3.0,978465565,3.350000,3.505391,3.288417,3.588809,3.427695,3.396904,3.433154
93850,599,4351,3.0,1498524542,3.000000,3.361111,3.032795,2.424717,3.180556,3.196953,2.954656
6187,42,2987,4.0,996262677,3.812500,3.207921,3.838729,3.836998,3.510210,3.523325,3.674037
12229,75,1610,4.0,1158989841,3.625000,3.582380,3.806743,3.338901,3.603690,3.694561,3.588256
...,...,...,...,...,...,...,...,...,...,...,...
57416,380,5048,2.0,1494268065,2.000000,3.551020,2.000000,3.254596,2.775510,2.775510,2.701404
67290,434,54272,3.5,1270606860,3.625000,3.990000,3.615817,3.494291,3.807500,3.802909,3.681277
33423,226,5989,4.5,1162428551,3.852941,3.668297,3.934037,3.901221,3.760619,3.801167,3.839124
98552,607,1320,3.0,963080497,3.045455,4.114672,3.036918,3.591904,3.580063,3.575795,3.447237


###### Calculating and comparing RMSE & MAE for different predctions

In [45]:
#RMSE for CB_Pred
#MAE for CB_Pred
CB_rmse = mean_squared_error(test.Rating, test.CB_Pred, squared=False)
CB_mae = mean_absolute_error(test.Rating, test.CB_Pred)

print(CB_rmse)
print(CB_mae)

0.7338745529056596
0.5330705103829799


In [46]:
#RMSE for Content_Pred
#MAE for Content_Pred

Content_rmse = mean_squared_error(test.Rating, test.Content_Pred, squared=False)
Content_mae = mean_absolute_error(test.Rating, test.Content_Pred)
print(Content_rmse)
print(Content_mae)

1.0842302136235102
0.8397419050800397


In [47]:
#RMSE for CB_Pred2
#MAE for CB_Pred2

CB_rmse2 = mean_squared_error(test.Rating, test.CB_Pred2, squared=False)
CB_mae2 = mean_absolute_error(test.Rating, test.CB_Pred2)
print(CB_rmse2)
print(CB_mae2)

0.5325017834510832
0.37810364757096804


In [48]:
#RMSE for NN_Pred
#MAE for NN_Pred

NN_rmse = mean_squared_error(test.Rating, test.NN_Pred, squared=False)
NN_mae = mean_absolute_error(test.Rating, test.NN_Pred)
print(NN_rmse)
print(NN_mae)

0.8668751364016396
0.6598234414538603


In [49]:
#RMSE for CB_CP_Mean_Pred
#MAE for CB_CP_Mean_Pred


CB_CP_Mean_rmse = mean_squared_error(test.Rating, test.CB_CP_Mean_Pred, squared=False)
CB_CP_Mean_mae = mean_absolute_error(test.Rating, test.CB_CP_Mean_Pred)
print(CB_CP_Mean_rmse)
print(CB_CP_Mean_mae)

0.8411056759833839
0.650161202396858


In [50]:
#RMSE for CB2_CP_Mean_Pred
#MAE for CB2_CP_Mean_Pred

CB2_CP_Mean_rmse = mean_squared_error(test.Rating, test.CB2_CP_Mean_Pred, squared=False)
CB2_CP_Mean_mae = mean_absolute_error(test.Rating, test.CB2_CP_Mean_Pred)
print(CB2_CP_Mean_rmse)
print(CB2_CP_Mean_mae)

0.7494992792373908
0.5808634411458878


In [51]:
#RMSE for Overall_Mean_Pred
#MAE for Overall_Mean_Pred


Overall_Mean_rmse = mean_squared_error(test.Rating, test.Overall_Mean_Pred, squared=False)
Overall_Mean_mae = mean_absolute_error(test.Rating, test.Overall_Mean_Pred)
print(Overall_Mean_rmse)
print(Overall_Mean_mae)

0.7249342618821245
0.55777146499523


###### Ranking Algorithm

In [72]:
# Defineing a ranking fuction to  (for Overall_Mean_Pred)
def get_ranking(user_id):
    dfuser_subset = test[test['User_id']==user_id]
    #Sort dataframe in descending order based on the CB2_CP_Mean_Pred ratings and slice top 25
    dfuser_subsetRecomm = dfuser_subset.sort_values(by=['CB2_CP_Mean_Pred'], ascending=False).head(25)
    #Sort dataframe in descending order based on the actual ratings and slice top 25
    dfuser_subsetActual = dfuser_subset.sort_values(by=['Rating'], ascending=False).head(25)
    return dfuser_subsetRecomm, dfuser_subsetActual
dfuser_subsetRecomm, dfuser_subsetActual = get_ranking(599)
#Get List of predicted and actual movie_id's
Predicted_list = dfuser_subsetRecomm['Movie_id'].tolist()
#Get corresponding predicted and actual movie names
Actual_list = dfuser_subsetActual['Movie_id'].tolist()
Predicted_Recommendations = generic_additional_data[generic_additional_data.Movie_id.isin(Predicted_list)]
Actual_Recommendations = generic_additional_data[generic_additional_data.Movie_id.isin(Actual_list)]
print("Predicted Recommendation")
display(Predicted_Recommendations)
print("Actual Recommendation")
display(Actual_Recommendations)
#Calculate recommendation accuracy
accuracy = ((len(set(Predicted_list).intersection(Actual_list)))/len(Actual_list))*100
print("Accuracy = ", accuracy, "%")

Predicted Recommendation


,Movie_id,Title,Genre
43,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
46,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
585,720,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
828,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
899,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Fantasy|Romance
902,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi


Actual Recommendation


,Movie_id,Title,Genre
99,112,Rumble in the Bronx (Hont faan kui) (1995),Action|Adventure|Comedy|Crime
257,296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
585,720,Wallace & Gromit: The Best of Aardman Animatio...,Adventure|Animation|Comedy
793,1036,Die Hard (1988),Action|Crime|Thriller
828,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller
863,1136,Monty Python and the Holy Grail (1975),Adventure|Comedy|Fantasy
898,1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi
902,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
911,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Sci-Fi
916,1215,Army of Darkness (1993),Action|Adventure|Comedy|Fantasy|Horror


Accuracy =  60.0 %


###### Accuracy of all users and Average Recommendation accuracy

In [68]:
from statistics import mean
def get_average_acc():
    users = test['User_id'].unique()
    acc = {}
    for u in users:
        dfuser_subset = test[test['User_id']==u]
        #Sort dataframe in descending order based on the CB2_CP_Mean_Pred ratings and slice top 25
        dfuser_subsetRecomm = dfuser_subset.sort_values(by=['CB2_CP_Mean_Pred'], ascending=False).head(25)
        #Sort dataframe in descending order based on the actual ratings and slice top 25
        dfuser_subsetActual = dfuser_subset.sort_values(by=['Rating'], ascending=False).head(25)
        #Get List of predicted and actual movie_id's
        Predicted_list = dfuser_subsetRecomm['Movie_id'].tolist()
        Actual_list = dfuser_subsetActual['Movie_id'].tolist()
        #print(u)
        acc[u] = ((len(set(Predicted_list).intersection(Actual_list)))/len(Actual_list))*100
    return acc
acc = get_average_acc()
acc1= pd.DataFrame.from_dict(data=acc, orient='index', columns=['Accuracy_per_user'])
display(acc1)
#Calculate and print overall recommendation accuracy.
print("Overall prediction accuracy : ", acc1['Accuracy_per_user'].mean())

,Accuracy_per_user
432,76.0
288,72.0
599,60.0
42,80.0
75,100.0
...,...
257,100.0
496,100.0
392,100.0
138,100.0


Overall prediction accuracy :  92.68852459016394


##### Hence we Understand how Hybrid Filter give better accuracy.